# Patient & PatientDemographic Merge and Validation 

Objective:
    - Merge the Patient_prepared.csv and PatientDemographic_prepared.csv files 
      into a single PatientDemographic_merged_prepared.csv
    - Keep only the ~100k Demographic sample (left join)
    - Standardize the Sex column => 'Male', 'Female', or 'Unknown'
    
Data Paths:
    - C:\Users\ProjectC4M\Documents\CPCSSN Datasets Care4Mind\New Extraction Feb 2025\prepared_data\PatientDemographic_prepared.csv
    - C:\Users\ProjectC4M\Documents\CPCSSN Datasets Care4Mind\New Extraction Feb 2025\prepared_data\Patient_prepared.csv
    - Output: 
      C:\Users\ProjectC4M\Documents\CPCSSN Datasets Care4Mind\New Extraction Feb 2025\prepared_data\PatientDemographic_merged_prepared.csv


In [3]:

# In[1]: Imports and Setup
import pandas as pd
import numpy as np

# File paths (edit if needed)
demo_file = r"C:\Users\ProjectC4M\Documents\CPCSSN Datasets Care4Mind\New Extraction Feb 2025\prepared_data\PatientDemographic_prepared.csv"
patient_file = r"C:\Users\ProjectC4M\Documents\CPCSSN Datasets Care4Mind\New Extraction Feb 2025\prepared_data\Patient_prepared.csv"
out_file = r"C:\Users\ProjectC4M\Documents\CPCSSN Datasets Care4Mind\New Extraction Feb 2025\prepared_data\PatientDemographic_merged_prepared.csv"

# In[2]: Load data
df_demo = pd.read_csv(demo_file, sep=",", dtype=str)
df_patient = pd.read_csv(patient_file, sep=",", dtype=str)

print(f"Loaded PatientDemographic: {df_demo.shape[0]} rows, {df_demo.shape[1]} columns")
print(f"Loaded Patient: {df_patient.shape[0]} rows, {df_patient.shape[1]} columns")

# In[3]: Define a helper to clean "Sex"
def clean_sex(val: str) -> str:
    """
    Convert any sex string to 'Male', 'Female', or 'Unknown'.
    """
    if pd.isnull(val):
        return "Unknown"
    val_lower = val.strip().lower()
    if val_lower in ["male", "m"]:
        return "Male"
    elif val_lower in ["female", "f"]:
        return "Female"
    else:
        return "Unknown"

# Standardize sex in both dataframes if the column exists
if "Sex" in df_demo.columns:
    df_demo["Sex"] = df_demo["Sex"].apply(clean_sex)

if "Sex" in df_patient.columns:
    df_patient["Sex"] = df_patient["Sex"].apply(clean_sex)

# In[4]: Merge on 'Patient_ID', using how='left' to keep only the sample from df_demo
df_merged = pd.merge(
    df_demo,
    df_patient,
    on="Patient_ID",
    how="left",  # left join => keep only patients from df_demo
    suffixes=("_demo", "_patient")
)

print(f"After merge: {df_merged.shape[0]} rows, {df_merged.shape[1]} columns")

# In[5]: If both dataframes had 'Sex', unify them into one column
if "Sex_demo" in df_merged.columns and "Sex_patient" in df_merged.columns:
    # Prefer the demographic side if not Unknown, else use patient
    df_merged["Sex"] = np.where(
        df_merged["Sex_demo"] != "Unknown",
        df_merged["Sex_demo"],
        df_merged["Sex_patient"]
    )
    df_merged.drop(["Sex_demo", "Sex_patient"], axis=1, inplace=True)
elif "Sex_demo" in df_merged.columns:
    df_merged.rename(columns={"Sex_demo": "Sex"}, inplace=True)
elif "Sex_patient" in df_merged.columns:
    df_merged.rename(columns={"Sex_patient": "Sex"}, inplace=True)

# In[6]: Save final merged result
df_merged.to_csv(out_file, index=False)
print(f"Merged data saved to: {out_file}")


Loaded PatientDemographic: 99989 rows, 16 columns
Loaded Patient: 352161 rows, 6 columns
After merge: 99989 rows, 21 columns
Merged data saved to: C:\Users\ProjectC4M\Documents\CPCSSN Datasets Care4Mind\New Extraction Feb 2025\prepared_data\PatientDemographic_merged_prepared.csv


## Check the Number of Unique Patient IDs

In [4]:

# In[7]: Confirm unique patient count
num_unique = df_merged["Patient_ID"].nunique()
print(f"Unique Patient_IDs in merged dataset: {num_unique}")

Unique Patient_IDs in merged dataset: 99986


# Validate Relationships to the 100K Sample
-----------------------------------------------------------------------
Goal:
   - Ensure that every table in 'prepared_data' only references the 100,000 
     patients in PatientDemographic_merged_prepared.csv
   - Report any out‐of‐range Patient_IDs found in each table (if any)
   - Optionally, also check Encounter_ID relationships 
     (e.g. in EncounterDiagnosis, Medication, Lab) referencing the Encounter table

In [5]:

import os

# 1) Load the 100k patient set
merged_file = r"C:\Users\ProjectC4M\Documents\CPCSSN Datasets Care4Mind\New Extraction Feb 2025\prepared_data\PatientDemographic_merged_prepared.csv"
df_merged = pd.read_csv(merged_file, dtype=str)
sample_patients = set(df_merged["Patient_ID"])

print(f"Number of unique patients in merged file: {len(sample_patients):,}\n")

# 2) Define which prepared tables to validate
prepared_dir = r"C:\Users\ProjectC4M\Documents\CPCSSN Datasets Care4Mind\New Extraction Feb 2025\prepared_data"
tables_to_check = [
    "Encounter_prepared.csv",
    "EncounterDiagnosis_prepared.csv",
    "FamilyHistory_prepared.csv",
    "HealthCondition_prepared.csv",
    "Lab_prepared.csv",
    "MedicalProcedure_prepared.csv",
    "Medication_prepared.csv",
    "Referral_prepared.csv",
    "RiskFactor_prepared.csv"
]

# 3) Check each table for invalid Patient_IDs
for fname in tables_to_check:
    fpath = os.path.join(prepared_dir, fname)
    if not os.path.exists(fpath):
        print(f"WARNING: file not found {fname}, skipping.")
        continue
    
    df_check = pd.read_csv(fpath, dtype=str)
    
    if "Patient_ID" not in df_check.columns:
        print(f"{fname}: No 'Patient_ID' column found, skipping check.")
        continue
    
    # Identify any rows where Patient_ID not in the 100K set
    df_invalid = df_check[~df_check["Patient_ID"].isin(sample_patients)]
    n_invalid = len(df_invalid)
    total = len(df_check)
    
    if n_invalid == 0:
        print(f"{fname}: OK - all {total:,} rows reference valid 100K patients.")
    else:
        print(f"{fname}: Found {n_invalid:,} rows (of {total:,}) with Patient_ID not in 100K sample.")
        # Optionally, save or inspect df_invalid for debug:
        # df_invalid.to_csv(f"{fname}_invalid_patients.csv", index=False)

print("\nPatient relationship checks completed.")

Number of unique patients in merged file: 99,986

Encounter_prepared.csv: OK - all 3,250,590 rows reference valid 100K patients.
EncounterDiagnosis_prepared.csv: OK - all 3,281,858 rows reference valid 100K patients.
FamilyHistory_prepared.csv: OK - all 82,731 rows reference valid 100K patients.
HealthCondition_prepared.csv: OK - all 722,806 rows reference valid 100K patients.
Lab_prepared.csv: OK - all 859 rows reference valid 100K patients.
MedicalProcedure_prepared.csv: OK - all 307,511 rows reference valid 100K patients.
Medication_prepared.csv: OK - all 905,303 rows reference valid 100K patients.
Referral_prepared.csv: OK - all 323,610 rows reference valid 100K patients.
RiskFactor_prepared.csv: OK - all 146,819 rows reference valid 100K patients.

Patient relationship checks completed.


the code checks for any row whose Patient_ID is not in our sampled set, and it found zero such rows in each table. That means the rows in those tables only link to the 99,986 patients we selected—so from a patient‐level perspective, the tables are consistent with the sampling.